In [70]:
from __future__ import unicode_literals
from __future__ import division

import json
import pandas as pd
import requests

from ast import literal_eval
from jinja2 import Template
from jsonschema import Draft4Validator
from pyspark.sql.types import TimestampType

pd.options.display.max_columns = None

ORIGINAL_BUCKET_NAME = 'safe-ucosp-2017'
TEST_BUCKET = 'telemetry-test-bucket/birdsarah-test'

N_FILES = 20000

In [2]:
bad_location_2 = "s3a://{}/bad_{}.parquet".format(TEST_BUCKET, N_FILES)
good_location_2 = "s3a://{}/good_{}.parquet".format(TEST_BUCKET, N_FILES)

In [3]:
bad_data = spark.read.parquet(bad_location_2).toPandas()

In [4]:
good_data = spark.read.parquet(good_location_2)

In [27]:
for val in bad_data.errors:
    print(val)

script_url:  not valid ||
script_line: webpack not valid ||
script_col: ///./node_modules/ not valid ||

script_url:  not valid ||
script_line: webpack not valid ||
script_col: ///./node_modules/ not valid ||

script_url:  not valid ||
script_line: webpack not valid ||
script_col: ///./node_modules/ not valid ||

script_url:  not valid ||
script_line: webpack not valid ||
script_col: ///./node_modules/ not valid ||

script_url:  not valid ||
script_line:  not valid ||

script_url:  not valid ||
script_line: webpack not valid ||
script_col: ///./node_modules/ not valid ||

script_url:  not valid ||
script_line: webpack not valid ||
script_col: ///./node_modules/ not valid ||

script_url:  not valid ||
script_line: webpack not valid ||
script_col: ///./node_modules/ not valid ||

script_url:  not valid ||
script_line: webpack not valid ||
script_col: ///./node_modules/ not valid ||

script_url:  not valid ||
script_line: http not valid ||
script_col: //adv.khan.co.kr/RealMedia/ads/adstre

In [6]:
bad_count = len(bad_data)
good_count = good_data.count()
print('{:.4%}'.format(bad_count / good_count))

0.0603%


In [7]:
good_data.agg({"call_id": "min"}).collect()

[Row(min(call_id)=u'1_0003cdd91a5b81c4d58654ca6c8e18b1c60ed6191c3cb849e37a51af.json__0')]

In [8]:
good_data.agg({"call_id": "max"}).collect()

[Row(max(call_id)=u'1_fffc766461372954c01be39f51c0ce49b9370e4d07bc892f296a87c9.json__6')]

In [9]:
http_location = good_data[good_data.location.startswith('http:')]
http_location_count = http_location.count()
https_location = good_data[good_data.location.startswith('https:')]
https_location_count = https_location.count()

In [11]:
print('{:,} rows'.format(good_count))
print('{:,} http rows'.format(http_location_count))
print('{:,} https rows'.format(https_location_count))
print('Missing: {}'.format(good_count - https_location_count - http_location_count))
print('Proportion https: {:.2%}'.format((https_location_count / good_count)))

1,138,232 rows
400,022 http rows
738,210 https rows
Missing: 0
Proportion https: 64.86%


In [12]:
mixed_content = https_location[~https_location.script_url.startswith('https')]

In [13]:
initial_char = mixed_content.script_url.substr(0, 1)
mixed_content.withColumn('initial_char', initial_char).groupBy('initial_char').count().show()

+------------+-----+
|initial_char|count|
+------------+-----+
|           l|    3|
|           0|   18|
|           m|   12|
|           f|   45|
|           _|  246|
|           e|   20|
|           C|    1|
|           /|  478|
|           w|   48|
|           i|   53|
|           r|  836|
|           t|   52|
|           s|   67|
+------------+-----+



In [14]:
good_data.groupBy('operation').count().show()

+---------+------+
|operation| count|
+---------+------+
|      set|145683|
|     call|264675|
|      get|727874|
+---------+------+



In [15]:
good_data.dtypes

[('argument_0', 'string'),
 ('argument_1', 'string'),
 ('argument_2', 'string'),
 ('argument_3', 'string'),
 ('argument_4', 'string'),
 ('argument_5', 'string'),
 ('argument_6', 'string'),
 ('argument_7', 'string'),
 ('argument_8', 'string'),
 ('arguments', 'string'),
 ('arguments_n_keys', 'bigint'),
 ('call_id', 'string'),
 ('call_stack', 'string'),
 ('crawl_id', 'bigint'),
 ('file_name', 'string'),
 ('func_name', 'string'),
 ('in_iframe', 'boolean'),
 ('location', 'string'),
 ('operation', 'string'),
 ('script_col', 'string'),
 ('script_line', 'string'),
 ('script_loc_eval', 'string'),
 ('script_url', 'string'),
 ('symbol', 'string'),
 ('time_stamp', 'timestamp'),
 ('value', 'string'),
 ('value_1000', 'string'),
 ('value_len', 'int'),
 ('valid', 'boolean'),
 ('errors', 'string')]

In [16]:
good_data.agg({"time_stamp": "min"}).show(1, False)

+-----------------------+
|min(time_stamp)        |
+-----------------------+
|2017-12-15 22:03:00.695|
+-----------------------+



In [17]:
good_data.agg({"time_stamp": "max"}).show(1, False)

+-----------------------+
|max(time_stamp)        |
+-----------------------+
|2017-12-17 01:26:50.584|
+-----------------------+



### Turns out there are some acceptable timestamps that we've rejected

In [23]:
time_stamp_rejects = bad_data[bad_data.errors.str.contains("time_stamp")]
time_stamp_rejects.head(2)

,argument_0,argument_1,argument_2,argument_3,argument_4,argument_5,argument_6,argument_7,argument_8,arguments,arguments_n_keys,call_id,call_stack,crawl_id,file_name,func_name,in_iframe,location,operation,script_col,script_line,script_loc_eval,script_url,symbol,time_stamp,value,value_1000,value_len,valid,errors
528,,,,,,,,,,,None,1_81c7ac09151998d58700669645e2dd69891532b068dd...,,1,1_81c7ac09151998d58700669645e2dd69891532b068dd...,,False,https://www.tripadvisor.co.uk/BusinessAdvantage,get,77,16,,https://static.tacdn.com/js3/tripadvisor-c-v23...,window.navigator.userAgent,2017-12-16T07:04:56+0000,Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko...,,,False,time_stamp: 2017-12-16T07:04:56+0000 not valid...
529,,,,,,,,,,,None,1_81c7ac09151998d58700669645e2dd69891532b068dd...,,1,1_81c7ac09151998d58700669645e2dd69891532b068dd...,,False,https://www.tripadvisor.co.uk/BusinessAdvantage,get,140,16,,https://static.tacdn.com/js3/tripadvisor-c-v23...,window.navigator.platform,2017-12-16T07:04:56+0000,Linux x86_64,,,False,time_stamp: 2017-12-16T07:04:56+0000 not valid...


In [28]:
time_stamp_rejects.file_name.unique()

array([u'1_81c7ac09151998d58700669645e2dd69891532b068dd552bf77e6bfb.json',
       u'1_678ed8d87c1e36a2fa5f429400a62161761658737c49bb3765ad7d50.json'], dtype=object)

In [54]:
symbol_counts = pd.read_csv('symbol_counts.csv', names=['symbol', 'count'])
with open('raw_data_schema.template', 'r') as f:
    schema_template = Template(f.read())
schema = literal_eval(
    schema_template.render(
        list_of_symbols=list(symbol_counts.symbol.values)
    )
)

def get_rows_from_s3(file_name):
    file_url = 'https://s3.amazonaws.com/{}/{}'.format(ORIGINAL_BUCKET_NAME, file_name)
    response = requests.get(file_url)
    assert response.status_code == 200
    fix_data = lambda x:  "[" + x[1:-1] + "]"
    data = fix_data(response.content)
    rows = json.loads(data)
    return rows

def validate_and_process_file(file_name):
    validator = Draft4Validator(schema)
    rows = get_rows_from_s3(file_name)

    for i, row in enumerate(rows):
        errors = ''
        valid = validator.is_valid(row)
        
        # If data was not valid, get the error information
        if valid is False:
            for error in validator.iter_errors(row):
                bad_value_truncated = error.instance[:75] + (error.instance[75:] and '...')
                errors += '{}: {} not valid ||\n'.format(
                    error.path[0], bad_value_truncated
                )
        
        row['file_name'] = '{}'.format(file_name)
        row['call_id'] = '{}__{}'.format(file_name, i)
        row['valid'] = valid
        row['errors'] = errors
        
        if valid is True:
            row = process_arguments(row)
        
    return rows

In [55]:
rows = validate_and_process_file('1_81c7ac09151998d58700669645e2dd69891532b068dd552bf77e6bfb.json')

In [56]:
rows[2]

{u'arguments': u'{}',
 u'arguments_n_keys': 0,
 u'call_id': u'1_81c7ac09151998d58700669645e2dd69891532b068dd552bf77e6bfb.json__2',
 u'call_stack': u'',
 u'crawl_id': 1,
 u'errors': u'',
 u'file_name': u'1_81c7ac09151998d58700669645e2dd69891532b068dd552bf77e6bfb.json',
 u'func_name': u'version<',
 u'in_iframe': False,
 u'location': u'https://www.tripadvisor.co.uk/BusinessAdvantage',
 u'operation': u'get',
 u'script_col': u'1543',
 u'script_line': u'10',
 u'script_loc_eval': u'',
 u'script_url': u'https://static.tacdn.com/js3/mootools-c-v22847647807b.js',
 u'symbol': u'window.navigator.plugins[Shockwave Flash].description',
 u'time_stamp': u'2017-12-16T07:04:56.069Z',
 u'valid': True,
 u'value': u'Shockwave Flash 28.0 r0'}

In [57]:
rows[3]

{u'arguments': u'{}',
 u'arguments_n_keys': 0,
 u'call_id': u'1_81c7ac09151998d58700669645e2dd69891532b068dd552bf77e6bfb.json__3',
 u'call_stack': u'',
 u'crawl_id': 1,
 u'errors': u'',
 u'file_name': u'1_81c7ac09151998d58700669645e2dd69891532b068dd552bf77e6bfb.json',
 u'func_name': u'',
 u'in_iframe': False,
 u'location': u'https://www.tripadvisor.co.uk/BusinessAdvantage',
 u'operation': u'get',
 u'script_col': u'77',
 u'script_line': u'16',
 u'script_loc_eval': u'',
 u'script_url': u'https://static.tacdn.com/js3/tripadvisor-c-v23806297793b.js',
 u'symbol': u'window.navigator.userAgent',
 u'time_stamp': u'2017-12-16T07:04:56+0000',
 u'valid': True,
 u'value': u'Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'}

In [72]:
fileRDD = sc.parallelize(['1_81c7ac09151998d58700669645e2dd69891532b068dd552bf77e6bfb.json'])
validated_rows = fileRDD.flatMap(validate_and_process_file)
validated_df = validated_rows.map(lambda x: (x.get('time_stamp', ""),)).toDF(('time_stamp',))
validated_df = validated_df.withColumn('time_stamp', validated_df.time_stamp.cast(TimestampType()))
validated_df.show(100, False)

+-----------------------+
|time_stamp             |
+-----------------------+
|2017-12-16 07:04:56.067|
|2017-12-16 07:04:56.068|
|2017-12-16 07:04:56.069|
|2017-12-16 07:04:56    |
|2017-12-16 07:04:56    |
|2017-12-16 07:04:56    |
|2017-12-16 07:04:56    |
|2017-12-16 07:04:56    |
|2017-12-16 07:04:56    |
|2017-12-16 07:04:56    |
|2017-12-16 07:04:56    |
|2017-12-16 07:04:56    |
|2017-12-16 07:04:56    |
|2017-12-16 07:04:56    |
|2017-12-16 07:04:56    |
|2017-12-16 07:04:56    |
|2017-12-16 07:04:56    |
|2017-12-16 07:04:56    |
|2017-12-16 07:04:56    |
|2017-12-16 07:04:56    |
|2017-12-16 07:04:56    |
|2017-12-16 07:04:56    |
|2017-12-16 07:04:56    |
|2017-12-16 07:04:56    |
|2017-12-16 07:04:56    |
|2017-12-16 07:04:56    |
|2017-12-16 07:04:56    |
|2017-12-16 07:04:56    |
|2017-12-16 07:04:56    |
|2017-12-16 07:04:56    |
|2017-12-16 07:04:56    |
|2017-12-16 07:04:56    |
|2017-12-16 07:04:56    |
|2017-12-16 07:04:56    |
|2017-12-16 07:04:56    |
|2017-12-16 

### Time to process

14 min for 20,000 rows - 8 workers

In [73]:
multiplier = 2059735 / 20000
multiplier

102.98675

In [79]:
estimated_time_to_complete = (14 * multiplier) / 60
'{:.0f} hours with 8 workers'.format(estimated_time_to_complete)

u'24 hours with 8 workers'

In [81]:
'{:.0f} hours with 30 workers'.format(estimated_time_to_complete / (30/8))

u'6 hours with 30 workers'

In [82]:
good_data.dtypes

[('argument_0', 'string'),
 ('argument_1', 'string'),
 ('argument_2', 'string'),
 ('argument_3', 'string'),
 ('argument_4', 'string'),
 ('argument_5', 'string'),
 ('argument_6', 'string'),
 ('argument_7', 'string'),
 ('argument_8', 'string'),
 ('arguments', 'string'),
 ('arguments_n_keys', 'bigint'),
 ('call_id', 'string'),
 ('call_stack', 'string'),
 ('crawl_id', 'bigint'),
 ('file_name', 'string'),
 ('func_name', 'string'),
 ('in_iframe', 'boolean'),
 ('location', 'string'),
 ('operation', 'string'),
 ('script_col', 'string'),
 ('script_line', 'string'),
 ('script_loc_eval', 'string'),
 ('script_url', 'string'),
 ('symbol', 'string'),
 ('time_stamp', 'timestamp'),
 ('value', 'string'),
 ('value_1000', 'string'),
 ('value_len', 'int'),
 ('valid', 'boolean'),
 ('errors', 'string')]

In [83]:
good_data.agg({"arguments_n_keys": "max"}).show(1, False)

+---------------------+
|max(arguments_n_keys)|
+---------------------+
|6                    |
+---------------------+



In [86]:
max_keys = good_data[good_data.arguments_n_keys == 6].toPandas()

In [87]:
max_keys

,argument_0,argument_1,argument_2,argument_3,argument_4,argument_5,argument_6,argument_7,argument_8,arguments,arguments_n_keys,call_id,call_stack,crawl_id,file_name,func_name,in_iframe,location,operation,script_col,script_line,script_loc_eval,script_url,symbol,time_stamp,value,value_1000,value_len,valid,errors
0,50,50,50,0,6.283185307179586,true,,,,"{""0"":50,""1"":50,""2"":50,""3"":0,""4"":6.283185307179...",6,1_294a4c2b123af5030649020e47d949319dc240aca847...,,1,1_294a4c2b123af5030649020e47d949319dc240aca847...,e.prototype.getCanvasFp,False,https://ck101.com/forum-3689-1.html,call,17893,1,,https://a.breaktime.com.tw/js/au-ac.js,CanvasRenderingContext2D.arc,2017-12-16 23:57:03.471,,,0,True,
1,100,50,50,0,6.283185307179586,true,,,,"{""0"":100,""1"":50,""2"":50,""3"":0,""4"":6.28318530717...",6,1_294a4c2b123af5030649020e47d949319dc240aca847...,,1,1_294a4c2b123af5030649020e47d949319dc240aca847...,e.prototype.getCanvasFp,False,https://ck101.com/forum-3689-1.html,call,17990,1,,https://a.breaktime.com.tw/js/au-ac.js,CanvasRenderingContext2D.arc,2017-12-16 23:57:03.474,,,0,True,
2,75,100,50,0,6.283185307179586,true,,,,"{""0"":75,""1"":100,""2"":50,""3"":0,""4"":6.28318530717...",6,1_294a4c2b123af5030649020e47d949319dc240aca847...,,1,1_294a4c2b123af5030649020e47d949319dc240aca847...,e.prototype.getCanvasFp,False,https://ck101.com/forum-3689-1.html,call,18088,1,,https://a.breaktime.com.tw/js/au-ac.js,CanvasRenderingContext2D.arc,2017-12-16 23:57:03.474,,,0,True,
3,75,75,75,0,6.283185307179586,true,,,,"{""0"":75,""1"":75,""2"":75,""3"":0,""4"":6.283185307179...",6,1_294a4c2b123af5030649020e47d949319dc240aca847...,,1,1_294a4c2b123af5030649020e47d949319dc240aca847...,e.prototype.getCanvasFp,False,https://ck101.com/forum-3689-1.html,call,18172,1,,https://a.breaktime.com.tw/js/au-ac.js,CanvasRenderingContext2D.arc,2017-12-16 23:57:03.475,,,0,True,
4,75,75,25,0,6.283185307179586,true,,,,"{""0"":75,""1"":75,""2"":25,""3"":0,""4"":6.283185307179...",6,1_294a4c2b123af5030649020e47d949319dc240aca847...,,1,1_294a4c2b123af5030649020e47d949319dc240aca847...,e.prototype.getCanvasFp,False,https://ck101.com/forum-3689-1.html,call,18203,1,,https://a.breaktime.com.tw/js/au-ac.js,CanvasRenderingContext2D.arc,2017-12-16 23:57:03.475,,,0,True,
5,190,-40,100,50,100,50,,,,"{""0"":190,""1"":-40,""2"":100,""3"":50,""4"":100,""5"":50}",6,1_381930ffb27a30dd209881b2ba58fff64bc0e9f216bf...,,1,1_381930ffb27a30dd209881b2ba58fff64bc0e9f216bf...,T/q<,False,http://igg-games.com/donate-share-game-support...,call,192,27,,http://c.adsco.re/,CanvasRenderingContext2D.bezierCurveTo,2017-12-15 23:31:59.464,,,0,True,
6,0,0,0.5,0,6.283185307179586,true,,,,"{""0"":0,""1"":0,""2"":0.5,""3"":0,""4"":6.2831853071795...",6,1_45ed8d8e5d7ae44f7df637a8065eeec682a0366fe90b...,,1,1_45ed8d8e5d7ae44f7df637a8065eeec682a0366fe90b...,_[E].arc,False,http://koubei.checheng.com/1868,call,20952,4,,http://static.checheng.com/common/car/js/echar...,CanvasRenderingContext2D.arc,2017-12-16 15:49:50.120,,,0,True,
7,0,0,0.5,0,6.283185307179586,true,,,,"{""0"":0,""1"":0,""2"":0.5,""3"":0,""4"":6.2831853071795...",6,1_45ed8d8e5d7ae44f7df637a8065eeec682a0366fe90b...,,1,1_45ed8d8e5d7ae44f7df637a8065eeec682a0366fe90b...,_[E].arc,False,http://koubei.checheng.com/1868,call,20952,4,,http://static.checheng.com/common/car/js/echar...,CanvasRenderingContext2D.arc,2017-12-16 15:49:50.125,,,0,True,
8,0,0,0.5,0,6.283185307179586,true,,,,"{""0"":0,""1"":0,""2"":0.5,""3"":0,""4"":6.2831853071795...",6,1_45ed8d8e5d7ae44f7df637a8065eeec682a0366fe90b...,,1,1_45ed8d8e5d7ae44f7df637a8065eeec682a0366fe90b...,_[E].arc,False,http://koubei.checheng.com/1868,call,20952,4,,http://static.checheng.com/common/car/js/echar...,CanvasRenderingContext2D.arc,2017-12-16 15:49:50.126,,,0,True,
9,0,0,0.5,0,6.283185307179586,true,,,,"{""0"":0,""1"":0,""2"":0.5,""3"":0,""4"":6.2831853071795...",6,1_45ed8d8e5d7ae44f7df637a8065eeec682a0366fe90b...,,1,1_45ed8d8e5d7ae44f7df637a8065eeec682a0366fe90b...,_[E].arc,False,http://koubei.checheng.com/18

In [88]:
good_data.groupBy('in_iframe').count().show()

+---------+------+
|in_iframe| count|
+---------+------+
|     true|229460|
|    false|908772|
+---------+------+



In [92]:
good_data.groupBy('symbol').count().orderBy('count', ascending=False).show(100, False)

+---------------------------------------------------------------------+------+
|symbol                                                               |count |
+---------------------------------------------------------------------+------+
|window.document.cookie                                               |341035|
|window.navigator.userAgent                                           |147308|
|window.Storage.getItem                                               |104671|
|window.localStorage                                                  |87653 |
|window.sessionStorage                                                |41690 |
|window.Storage.setItem                                               |39289 |
|window.Storage.removeItem                                            |26465 |
|window.name                                                          |23416 |
|CanvasRenderingContext2D.fillStyle                                   |19133 |
|window.navigator.plugins[Shockwave Flash].descripti

In [94]:
locations = good_data.groupBy('location').count()
print(locations.count())

19997


In [95]:
file_names = good_data.groupBy('file_name').count()
print(file_names.count())

19997
